In [4]:
import pandas as pd
from pathlib import Path

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 3000)

download_path = Path.home() / "Downloads"  
OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet'

df_OHLCV = pd.read_parquet(OHLCV_file_path, engine='pyarrow')

In [5]:
df_OHLCV.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 375500 entries, ('A', Timestamp('2024-09-24 00:00:00')) to ('ZWS', Timestamp('2025-09-23 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Open   375500 non-null  float64
 1   Adj High   375500 non-null  float64
 2   Adj Low    375500 non-null  float64
 3   Adj Close  375500 non-null  float64
 4   Volume     375500 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 15.8+ MB


In [6]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pprint 

def plot_interactive_performers_widget(df_ohlcv, 
                                       default_metric='Price',
                                       default_period='1Y', 
                                       default_rank_start=1,
                                       default_rank_end=10):
    """
    Creates a robust interactive plot using ipywidgets for state management.
    Metrics include Price, Sharpe, and Return/ATR.
    (Final corrected version).
    """
    # --- 1. Data Preparation: Reshape & Calculate Intermediate Values ---
    print("Reshaping data and calculating intermediate metrics (ATR)...")
    
    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("The input DataFrame does not have the expected 'long' format with a (Ticker, Date) MultiIndex.")
    
    df_close = df_ohlcv['Adj Close'].unstack(level=0)
    df_high = df_ohlcv['Adj High'].unstack(level=0)
    df_low = df_ohlcv['Adj Low'].unstack(level=0)
    df_close.index = pd.to_datetime(df_close.index)
    df_high.index = pd.to_datetime(df_high.index)
    df_low.index = pd.to_datetime(df_low.index)

    high_low = df_high - df_low
    high_prev_close = abs(df_high - df_close.shift(1))
    low_prev_close = abs(df_low - df_close.shift(1))
    tr = np.maximum(high_low, np.maximum(high_prev_close, low_prev_close))
    df_atr = tr.ewm(alpha=1/14, adjust=False).mean()
    
    print("Data successfully reshaped and ATR calculated.")
    
    # --- 2. Pre-computation Phase ---
    print("Starting pre-computation for all metrics across all periods...")
    end_date = df_close.index.max()
    metrics = ['Price', 'Sharpe', 'Return/ATR']
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    all_rankings = {metric: {} for metric in metrics}
    period_normalized_data = {}
    for label, start_date in periods.items():
        period_close = df_close.loc[start_date:end_date]
        period_atr = df_atr.loc[start_date:end_date]
        if period_close.empty or len(period_close) < 2: continue
        first_prices, last_prices = period_close.bfill().iloc[0], period_close.ffill().iloc[-1]
        all_rankings['Price'][label] = (last_prices / first_prices).dropna().sort_values(ascending=False).index.tolist()
        daily_returns = period_close.pct_change()
        mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
        sharpe_ratio = (mean_returns / std_returns).fillna(0)
        all_rankings['Sharpe'][label] = (sharpe_ratio * np.sqrt(252)).sort_values(ascending=False).index.tolist()
        mean_atr = period_atr.mean()
        return_on_atr = (mean_returns / mean_atr).fillna(0)
        all_rankings['Return/ATR'][label] = return_on_atr.sort_values(ascending=False).index.tolist()
        period_normalized_data[label] = period_close.div(first_prices)
    print("Pre-computation finished.")

    # --- 3. Create ipywidgets Controls & Data Holder ---
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    max_traces = 50 
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    period_dropdown = widgets.Dropdown(options=list(periods.keys()), value=default_period, description='Period:')
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    _currently_displayed_tickers = []

    # --- 4. Create a FigureWidget ---
    fig = go.FigureWidget()
    for i in range(max_traces):
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

    # --- 5. Define the Update Logic ---
    def update_plot(button_click):
        metric, period, rank_start, rank_end = metric_dropdown.value, period_dropdown.value, rank_start_dropdown.value, rank_end_dropdown.value
        ticker_list_output.clear_output()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be less than or equal to 'Rank End'.")
            return
        ranked_tickers = all_rankings[metric].get(period, [])
        tickers_to_display = ranked_tickers[rank_start-1:rank_end]
        norm_data = period_normalized_data.get(period)

        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                
                # --- THIS IS THE CORRECTED LOGIC ---
                # First, check if there's a ticker available for this trace index
                if i < len(tickers_to_display) and norm_data is not None:
                    ticker = tickers_to_display[i]
                    # Second, check if that ticker actually has data in the norm_data frame
                    if ticker in norm_data.columns:
                        trace.x, trace.y, trace.name = norm_data[ticker].index, norm_data[ticker], ticker
                        trace.visible, trace.showlegend = True, True
                    else: # Handle cases where a ticker was ranked but has no data
                        trace.visible, trace.showlegend = False, False
                else: # If no ticker for this trace index, hide it
                    trace.visible, trace.showlegend = False, False

        _currently_displayed_tickers.clear()
        _currently_displayed_tickers.extend(tickers_to_display)
        with ticker_list_output:
            print("Currently Displayed Tickers:")
            pprint.pprint(tickers_to_display, width=120, compact=True) 
    
    # --- 6. Set up Initial View and Layout ---
    update_plot(None)
    fig.update_layout(title_text='Top Performing Tickers (Normalized to 1)', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    
    # --- 7. Display the UI ---
    controls = widgets.HBox([metric_dropdown, period_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
    ui_container = widgets.VBox([controls, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    return _currently_displayed_tickers

### **J. Welles Wilder Jr. definition ATR that captures overnight gaps.**

Let's be precise:

*   **We WILL calculate** the **True Range (TR)** for each day, which is the maximum of:
    1.  `(Today's High - Today's Low)`
    2.  `abs(Today's High - Yesterday's Close)`
    3.  `abs(Today's Low - Yesterday's Close)`
*   Then, we will typically calculate the **Average True Range (ATR)**, which is a **Exponential Moving Average** of this True Range value. This is the standard industry indicator for volatility that includes intraday and overnight movement.

### **Return/ATR Calculation**

You are correct that the ATR calculation should be tied to the `Period` dropdown, but here's how we'll do it in a standard, robust way:

**Step 1: Pre-computation (Done once for all stocks)**

First, for the entire historical dataset, we will calculate a standard **14-day Average True Range (ATR)**. This is the industry standard lookback period defined by Wilder. This step creates a new, continuous ATR data series for every single stock, just like we have an 'Adj Close' series.

**Step 2: On-the-Fly Calculation (When you click "Update Chart")**

This is where your interpretation comes in. When you make your selections in the dropdowns (e.g., `Metric: Return/ATR`, `Period: 3M`):

1.  We slice the **daily returns** data to get only the last **3 months**.
2.  We also slice the pre-computed **14-day ATR** data to get only the last **3 months**.
3.  We then calculate:
    *   **Numerator:** The `mean()` of the daily returns over those 3 months.
    *   **Denominator:** The `mean()` of the ATR values over those 3 months.
4.  The final score for the new metric is `Numerator / Denominator`.

**In short: The `Period` dropdown defines the window of data we analyze, not the lookback period of the ATR indicator itself.**

This is the best practice because it ensures we are always comparing apples to apples. We're using a consistent measure of volatility (14-day ATR) and seeing how it behaves over different analysis windows (1D, 3M, 1Y, etc.).

In [ ]:
# --- NEW: Initialize the variable before the call ---
plotted_tickers = [] 

# Call the final function and capture the returned list
# The 'plotted_tickers' list will be updated in place every time you click the button.
plotted_tickers = plot_interactive_performers_widget(
    df_OHLCV, 
    default_metric='Sharpe', 
    default_period='1Y', 
    default_rank_start=20, 
    default_rank_end=40
)

# print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
# print(plotted_tickers)

Reshaping data and calculating intermediate metrics (ATR)...
Data successfully reshaped and ATR calculated.
Starting pre-computation for all metrics across all periods...
Pre-computation finished.


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'ESLT',
              'showlegend': True,
              'type': 'scatter',
              'uid': '693b0c68-888c-4870-b29a-2125eef0d400',
              'visible': True,
              'x': array([datetime.datetime(2024, 9, 24, 0, 0),
                          datetime.datetime(2024, 9, 25, 0, 0),
                          datetime.datetime(2024, 9, 26, 0, 0), ...,
                          datetime.datetime(2025, 9, 19, 0, 0),
                          datetime.datetime(2025, 9, 22, 0, 0),
                          datetime.datetime(2025, 9, 23, 0, 0)], dtype=object),
              'y': array([1.        , 0.99948651, 1.00705402, ..., 2.53099099, 2.58804552,
                          2.58835673])},
             {'mode': 'lines',
              'name': 'SPSB',
              'showlegend': True,
              'type': 'scatter',
              'uid': '671c86c6-295a-4c81-a6ba-eb34212ab2ab',
              'visible': True,
        

In [16]:
print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
print(plotted_tickers)


--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---
['NIO', 'HL', 'UWMC', 'EQX', 'MFG', 'BBD', 'GRAB', 'QS', 'EBR', 'CDE', 'NMR', 'IAG', 'CX', 'AEG', 'SAN', 'LYG', 'WBD', 'SBSW', 'FPE', 'VALE']


### Plot Tickers

In [46]:
# --- 2. Define the list of tickers you want to keep ---
tickers_to_plot = ['SPY', 'VGT', 'QQQ', 'NVDA', 'GOOG', 'META', 'ORCL', 'APP', 'U', 'B', 'NIO',
                   'HL', 'UWMC', 'EQX', 'MFG', 'BBD', 'GRAB', 'QS', 'EBR', 'CDE', 'NMR', 'IAG',
                    'CX', 'AEG', 'SAN', 'GLD', 'IAUM', 'IBIT', 'MSTR',
                   'LYG', 'WBD', 'SBSW', 'FPE', 'VALE']



# --- 3. The Efficient Filtering Code ---
# This single line performs a fast, index-based selection.
# The result preserves the original MultiIndex structure and format.
filtered_df = df_OHLCV.loc[tickers_to_plot]


# --- 4. Verification ---
print("Filtered DataFrame created. Shape:", filtered_df.shape)
print("Filtered DataFrame Index Levels:", filtered_df.index.names)

# Verify that only the desired tickers are present
remaining_tickers = filtered_df.index.get_level_values('Ticker').unique().tolist()
print("Tickers remaining in filtered_df:", remaining_tickers)
assert set(tickers_to_plot) == set(remaining_tickers)

print("\nAll checks passed. The format is preserved correctly.")
display(filtered_df.head())

Filtered DataFrame created. Shape: (8500, 5)
Filtered DataFrame Index Levels: ['Ticker', 'Date']
Tickers remaining in filtered_df: ['SPY', 'VGT', 'QQQ', 'NVDA', 'GOOG', 'META', 'ORCL', 'APP', 'U', 'B', 'NIO', 'HL', 'UWMC', 'EQX', 'MFG', 'BBD', 'GRAB', 'QS', 'EBR', 'CDE', 'NMR', 'IAG', 'CX', 'AEG', 'SAN', 'GLD', 'IAUM', 'IBIT', 'MSTR', 'LYG', 'WBD', 'SBSW', 'FPE', 'VALE']

All checks passed. The format is preserved correctly.


Adj Open  Adj High  Adj Low  Adj Close    Volume
Ticker Date                                                        
SPY    2024-09-24   563.628   564.497  560.782    564.438  47374742
       2024-09-25   564.280   565.021  562.077    563.193  38895797
       2024-09-26   567.481   567.807  563.055    565.426  48923643
       2024-09-27   566.503   567.323  563.568    564.606  42612741
       2024-09-30   563.568   567.481  561.257    566.868  64330101

In [ ]:
# --- NEW: Initialize the variable before the call ---
plotted_tickers = [] 

# Call the final function and capture the returned list
# The 'plotted_tickers' list will be updated in place every time you click the button.
plotted_tickers = plot_interactive_performers_widget(
    filtered_df, 
    default_metric='Sharpe', 
    default_period='1Y', 
    default_rank_start=20, 
    default_rank_end=40
)

# print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
# print(plotted_tickers)

Reshaping data and calculating intermediate metrics (ATR)...
Data successfully reshaped and ATR calculated.
Starting pre-computation for all metrics across all periods...
Pre-computation finished.


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'EQX',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f8290fad-8196-40d8-932a-75711ce5af70',
              'visible': True,
              'x': array([datetime.datetime(2024, 9, 24, 0, 0),
                          datetime.datetime(2024, 9, 25, 0, 0),
                          datetime.datetime(2024, 9, 26, 0, 0), ...,
                          datetime.datetime(2025, 9, 19, 0, 0),
                          datetime.datetime(2025, 9, 22, 0, 0),
                          datetime.datetime(2025, 9, 23, 0, 0)], dtype=object),
              'y': array([1.        , 0.99530516, 1.01095462, ..., 1.69014085, 1.71205008,
                          1.68231612])},
             {'mode': 'lines',
              'name': 'U',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e5752e8a-58ec-433f-aef4-9561d32f0be8',
              'visible': True,
            

In [53]:
print(plotted_tickers)

['BBD', 'LYG', 'SAN', 'SBSW', 'EBR', 'MFG', 'HL', 'CDE', 'EQX', 'AEG', 'IAG', 'QS', 'CX', 'NIO', 'NMR', 'GRAB', 'WBD', 'B', 'VALE', 'UWMC', 'FPE', 'IAUM', 'U', 'ORCL', 'IBIT', 'GLD', 'NVDA', 'GOOG', 'APP', 'QQQ', 'SPY', 'VGT', 'META', 'MSTR']


### Load df_finviz

In [49]:
finviz_file_path = r'C:\Users\ping\Files_win10\python\py311\stocks\data\2025-09-23_df_finviz_merged_stocks_etfs.parquet'

In [ ]:
finviz_file_path = r'C:\Users\ping\Files_win10\python\py311\stocks\data\2025-09-23_df_finviz_merged_stocks_etfs.parquet'
df_finviz = pd.read_parquet(finviz_file_path, engine='pyarrow')

print(f'df_finviz :\n{df_finviz }')

df_finviz :
       No.                                        Company               Index                  Sector                        Industry Country Exchange                                               Info  MktCap AUM, M  Rank  Market Cap, M    P/E  Fwd P/E   PEG    P/S    P/B    P/C  P/FCF  Book/sh  Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %    EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %  Sales, M  Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M  Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M  ROA %   ROE %  ROIC %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf 3D %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %  Beta   ATR  ATR/Price %  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %  50D Hig

In [56]:
print(df_finviz.loc[plotted_tickers])

       No.                                          Company               Index                  Sector                          Industry         Country Exchange                                               Info  MktCap AUM, M  Rank  Market Cap, M     P/E  Fwd P/E    PEG     P/S     P/B      P/C   P/FCF  Book/sh  Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %    EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %   Sales, M  Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M  Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M  ROA %   ROE %  ROIC %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf 3D %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %  Beta    ATR  ATR/Price %  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %

### Verification Workflow Example

In [14]:
def inspect_ticker_data(df_ohlcv, ticker, period):
    """
    Provides a detailed, transparent breakdown of performance metric calculations for a single ticker over a specific period.
    
    Returns a pandas DataFrame containing the original OHLCV data plus all intermediate columns
    used to calculate Price, Sharpe, and Return/ATR metrics. This serves as a "calculation worksheet"
    for easy verification.
    """
    # 1. Initial Data Slicing and Validation
    if ticker not in df_ohlcv.index.get_level_values(0):
        print(f"Error: Ticker '{ticker}' not found in the DataFrame.")
        return
        
    ticker_df = df_ohlcv.loc[ticker].copy()
    ticker_df.index = pd.to_datetime(ticker_df.index)
    end_date = ticker_df.index.max()
    
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    
    if period not in periods:
        print(f"Error: Period '{period}' is not a valid option.")
        return
        
    start_date = periods[period]
    period_df = ticker_df.loc[start_date:end_date].copy()
    
    if len(period_df) < 2:
        print(f"Not enough data for ticker '{ticker}' in period '{period}'.")
        return

    # 2. Add Calculation Columns to the DataFrame
    # --- Price Performance Columns ---
    start_price = period_df['Adj Close'].bfill().iloc[0]
    period_df['Normalized Price'] = period_df['Adj Close'] / start_price

    # --- Sharpe Ratio Columns ---
    period_df['Daily Return'] = period_df['Adj Close'].pct_change()

    # --- Return/ATR Columns (matches the main function's logic) ---
    period_df['Prev Close'] = period_df['Adj Close'].shift(1)
    period_df['High-Low'] = period_df['Adj High'] - period_df['Adj Low']
    period_df['High-PrevClose'] = abs(period_df['Adj High'] - period_df['Prev Close'])
    period_df['Low-PrevClose'] = abs(period_df['Adj Low'] - period_df['Prev Close'])
    period_df['True Range'] = period_df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    # Use the same ewm parameters as the main plotting function for consistency
    period_df['ATR'] = period_df['True Range'].ewm(alpha=1/14, adjust=False).mean()

    # 3. Calculate Final Metrics for Summary Printout
    # Price
    end_price = period_df['Adj Close'].ffill().iloc[-1]
    normalized_price = end_price / start_price
    
    # Sharpe
    mean_daily_return = period_df['Daily Return'].mean()
    std_daily_return = period_df['Daily Return'].std()
    daily_sharpe = 0 if std_daily_return == 0 else mean_daily_return / std_daily_return
    annualized_sharpe = daily_sharpe * np.sqrt(252)

    # Return/ATR
    mean_atr = period_df['ATR'].mean()
    return_on_atr = 0 if mean_atr == 0 else mean_daily_return / mean_atr

    # 4. Display Summary and DataFrame
    display(Markdown(f"### Sanity Check for Ticker: `{ticker}` | Period: `{period}`"))
    
    display(Markdown("**1. Price Performance Calculation:**"))
    print(f"   - Start Date ({period_df.index.min().date()}): {start_price:,.2f}")
    print(f"   - End Date   ({period_df.index.max().date()}): {end_price:,.2f}")
    print(f"   - Final Normalized Price: ({end_price:,.2f} / {start_price:,.2f}) = {normalized_price:.4f}\n")

    display(Markdown("**2. Sharpe Ratio Calculation:**"))
    print(f"   - Number of Trading Days: {len(period_df)}")
    print(f"   - Mean of Daily Returns: {mean_daily_return:.6f}")
    print(f"   - Std Dev of Daily Returns: {std_daily_return:.6f}")
    print(f"   - Annualized Sharpe Ratio: {annualized_sharpe:.4f}\n")
    
    display(Markdown("**3. Return/ATR Calculation:**"))
    print(f"   - Mean of Daily Returns: {mean_daily_return:.6f}")
    print(f"   - Mean ATR (14-day EWM): {mean_atr:.6f}")
    print(f"   - Return / Mean ATR: {return_on_atr:.6f}\n")

    display(Markdown("**4. Underlying Data and Calculation Worksheet:**"))
    # Define a clean column order for the final display
    display_cols = [
        'Adj Open', 'Adj High', 'Adj Low', 'Adj Close', # Original Data
        'Normalized Price', 'Daily Return',             # Metric Columns
        'Prev Close', 'True Range', 'ATR'               # ATR Intermediates
    ]
    # Filter for columns that actually exist to avoid errors if some are missing
    display_cols = [col for col in display_cols if col in period_df.columns]
    with pd.option_context('display.max_rows', 15):
        display(period_df[display_cols])
    
    return period_df

In [23]:
# --- 1. Run the Main Plot to Get a Ranked List ---
# We'll use Sharpe over 1 Year and look at the top 10 performers.
_METRIC = 'Return/ATR'
_PERIOD = '3M'
_RANK_START = 1
_RANK_END = 10

print(f"--- Running main widget to get the Top {_RANK_END} tickers for '{_METRIC}' over '{_PERIOD}' ---")
# The UI will be displayed. Make sure to click "Update Chart" if defaults are different.
plotted_tickers = plot_interactive_performers_widget(
    df_OHLCV, 
    default_metric=_METRIC, 
    default_period=_PERIOD, 
    default_rank_start=_RANK_START, 
    default_rank_end=_RANK_END
)

# --- 2. Perform the Verification Check ---
print("\n" + "="*80)
print("--- STARTING VERIFICATION PROCESS ---")
print("="*80 + "\n")

# We need at least two tickers to compare. Let's check for the 1st and 5th.
if len(plotted_tickers) >= 5:
    ticker_to_check_1 = plotted_tickers[0]  # The #1 ranked ticker
    ticker_to_check_2 = plotted_tickers[4]  # The #5 ranked ticker

    print(f"\n--- Verifying Rank #1: {ticker_to_check_1} ---")
    inspection_df_1 = inspect_ticker_data(
        df_OHLCV, 
        ticker=ticker_to_check_1, 
        period=_PERIOD
    )

    print(f"\n--- Verifying Rank #5: {ticker_to_check_2} ---")
    inspection_df_2 = inspect_ticker_data(
        df_OHLCV, 
        ticker=ticker_to_check_2, 
        period=_PERIOD
    )
    
    # You can now manually compare the printed summaries above.
    # For Sharpe, look at the "Annualized Sharpe Ratio".
    # For Price, look at the "Final Normalized Price".
    # For Return/ATR, look at the "Return / Mean ATR".
    # The value for the first ticker should be greater than the value for the second.

else:
    print(f"Could not perform verification check: The plotter returned fewer than 5 tickers ({len(plotted_tickers)}).")

print("\n" + "="*80)
print("--- VERIFICATION PROCESS COMPLETE ---")
print("="*80)

--- Running main widget to get the Top 10 tickers for 'Return/ATR' over '3M' ---
Reshaping data and calculating intermediate metrics (ATR)...
Data successfully reshaped and ATR calculated.
Starting pre-computation for all metrics across all periods...
Pre-computation finished.


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'NIO',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6e7da9f9-1588-4b8e-ace8-99af94009760',
              'visible': True,
              'x': array([datetime.datetime(2025, 6, 23, 0, 0),
                          datetime.datetime(2025, 6, 24, 0, 0),
                          datetime.datetime(2025, 6, 25, 0, 0),
                          datetime.datetime(2025, 6, 26, 0, 0),
                          datetime.datetime(2025, 6, 27, 0, 0),
                          datetime.datetime(2025, 6, 30, 0, 0),
                          datetime.datetime(2025, 7, 1, 0, 0),
                          datetime.datetime(2025, 7, 2, 0, 0),
                          datetime.datetime(2025, 7, 3, 0, 0),
                          datetime.datetime(2025, 7, 7, 0, 0),
                          datetime.datetime(2025, 7, 8, 0, 0),
                          datetime.datetime(2025, 7, 9, 0, 0),
    


--- STARTING VERIFICATION PROCESS ---


--- Verifying Rank #1: NIO ---


### Sanity Check for Ticker: `NIO` | Period: `3M`

**1. Price Performance Calculation:**

...

**3. Return/ATR Calculation:**

   - Mean of Daily Returns: 0.003550
   - Mean ATR (14-day EWM): 0.108315
   - Return / Mean ATR: 0.032773



**4. Underlying Data and Calculation Worksheet:**

,Adj Open,Adj High,Adj Low,Adj Close,Normalized Price,Daily Return,Prev Close,True Range,ATR
Date,,,,,,,,,
2025-06-23,5.35,5.43,5.35,5.42,1.000000,NaN,NaN,0.08,0.080000
2025-06-24,5.57,5.62,5.57,5.60,1.033210,0.033210,5.42,0.20,0.088571
2025-06-25,5.49,5.53,5.49,5.51,1.016605,-0.016071,5.60,0.11,0.090102
2025-06-26,5.57,5.62,5.57,5.60,1.033210,0.016334,5.51,0.11,0.091523
2025-06-27,5.57,5.58,5.53,5.56,1.025830,-0.007143,5.60,0.07,0.089986
...,...,...,...,...,...,...,...,...,...
2025-09-17,6.56,6.63,6.53,6.56,1.210332,-0.001522,6.57,0.10,0.124442
2025-09-18,6.50,6.60,6.50,6.58,1.214022,0.003049,6.56,0.10,0.122696
2025-09-19,6.64,6.65,6.60,6.64,1.225092,0.009119,6.58,0.07,0.118932



--- VERIFICATION PROCESS COMPLETE ---


In [29]:
import os
# safe, platform-independent way
inspection_df_1.to_csv(os.path.join(download_path, f'{ticker_to_check_1}.csv'))
inspection_df_2.to_csv(os.path.join(download_path, f'{ticker_to_check_2}.csv'))

In [ ]:
cutoff = pd.Timestamp('today') - pd.DateOffset(months=3)
last_3m = df_OHLCV.loc[df_OHLCV.index >= cutoff].copy()
last_3m

In [37]:
df_OHLCV.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 375500 entries, ('A', Timestamp('2024-09-24 00:00:00')) to ('ZWS', Timestamp('2025-09-23 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Open   375500 non-null  float64
 1   Adj High   375500 non-null  float64
 2   Adj Low    375500 non-null  float64
 3   Adj Close  375500 non-null  float64
 4   Volume     375500 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 23.9+ MB


In [43]:
df_OHLCV.sort_index(inplace=True)
df_OHLCV.loc[(ticker_to_check_1, slice('2025-06-23', '2025-09-23')), :]

Adj Open  Adj High  Adj Low  Adj Close     Volume
Ticker Date                                                         
NIO    2025-06-23      3.40      3.45     3.34       3.43   28943200
       2025-06-24      3.46      3.57     3.41       3.50   33481300
       2025-06-25      3.57      3.57     3.44       3.47   21509500
       2025-06-26      3.46      3.48     3.40       3.42   31146400
       2025-06-27      3.42      3.55     3.40       3.46   63720200
       2025-06-30      3.49      3.49     3.41       3.43   24598900
       2025-07-01      3.47      3.57     3.45       3.51   46064800
       2025-07-02      3.50      3.54     3.47       3.48   22912600
       2025-07-03      3.50      3.55     3.47       3.51   15645500
       2025-07-07      3.44      3.47     3.39       3.41   31339200
       2025-07-08      3.43      3.52     3.38       3.50   43515600
       2025-07-09      3.52      3.59     3.46       3.48   26907300
       2025-07-10      3.55      3.70     3.49       3.69   74015200
       2025-07-11      3.75      4.05     3.70       3.90  141527800
       2025-07-14      4.11      4.22     4.02       4.17  122870700
       2025-07-15      4.25      4.28     4.13       4.25   66259200
       2025-07-16      4.19      4.19     3.96       4.11   69862800
       2025-07-17      4.13      4.49     4.11       4.40  116140100
       2025-07-18      4.38      4.57     4.28       4.39   78373600
       2025-07-21      4.47      4.67     4.46       4.52   58884000
       2025-07-22      4.64      5.09     4.64       5.01  133091900
       2025-07-23      5.19      5.21     4.87       4.92  108246500
       2025-07-24      4.92      4.95     4.70       4.83   63260400
       2025-07-25      4.84      5.09     4.79       4.92   60712300
       2025-07-28      4.99      5.01     4.82       4.83   42322400
       2025-07-29      4.82      4.84     4.68       4.71   38397400
       2025-07-30      4.60      4.61     4.51       4.51   40824100
       2025-07-31      4.68      4.98     4.46       4.87  131207100
       2025-08-01      4.74      5.06     4.73       5.01   81680900
       2025-08-04      4.89      4.90     4.58       4.60   92178900
       2025-08-05      4.66      4.67     4.46       4.50   55676400
       2025-08-06      4.59      4.67     4.52       4.61   55025100
       2025-08-07      4.62      4.72     4.59       4.68   27399000
       2025-08-08      4.73      4.87     4.71       4.84   36036300
       2025-08-11      4.96      5.01     4.85       4.91   36250500
       2025-08-12      4.74      4.74     4.45       4.47  106157200
       2025-08-13      4.61      4.62     4.54       4.62   35804200
       2025-08-14      4.50      4.51     4.37       4.45   43446300
       2025-08-15      4.56      4.82     4.55       4.81   69021000
       2025-08-18      4.91      4.97     4.81       4.87   47355300
       2025-08-19      4.94      5.23     4.93       5.07   90447700
       2025-08-20      5.01      5.14     4.97       5.07   46063500
       2025-08-21      5.24      5.68     5.24       5.54  147125700
       2025-08-22      5.87      6.58     5.78       6.34  234464700
       2025-08-25      6.70      6.73     6.07       6.09  156581500
       2025-08-26      6.39      6.83     6.38       6.70  146133100
       2025-08-27      6.50      6.62     6.31       6.34   89801700
       2025-08-28      6.53      6.54     6.26       6.51   74896800
       2025-08-29      6.44      6.51     6.26       6.38   57629700
       2025-09-02      6.24      6.64     6.06       6.58  123577100
       2025-09-03      6.38      6.40     6.04       6.32   89712400
       2025-09-04      6.07      6.19     5.97       6.13   72831200
       2025-09-05      6.18      6.23     5.92       5.95   59299500
       2025-09-08      6.05      6.23     5.95       6.17   81801400
       2025-09-09      6.06      6.32     6.05       6.28   54499200
       2025-09-10      5.74      5.78     5.60       5.72  237836500
       2025-09-11

In [19]:
# --- Example Usage after running the main plotting cell ---

# Let's say the main plot is showing tickers for 'Return/ATR' over '6M'
_period = '3M' 

# Pick the top-ranked ticker from the list returned by the main function
if plotted_tickers:
    _ticker = plotted_tickers[0] 

    # Call the new inspection function
    inspection_df = inspect_ticker_data(
        df_OHLCV, # Use the same dataframe you passed to the plot
        ticker=_ticker, 
        period=_period
    )
else:
    print("Run the main plotting widget first to populate the 'plotted_tickers' list.")

### Sanity Check for Ticker: `NIO` | Period: `3M`

**1. Price Performance Calculation:**

   - Start Date (2025-06-23): 3.43
   - End Date   (2025-09-23): 6.93
   - Final Normalized Price: (6.93 / 3.43) = 2.0204



**2. Sharpe Ratio Calculation:**

   - Number of Trading Days: 65
   - Mean of Daily Returns: 0.012151
   - Std Dev of Daily Returns: 0.047697
   - Annualized Sharpe Ratio: 4.0441



**3. Return/ATR Calculation:**

   - Mean of Daily Returns: 0.012151
   - Mean ATR (14-day EWM): 0.258025
   - Return / Mean ATR: 0.047093



**4. Underlying Data and Calculation Worksheet:**

,Adj Open,Adj High,Adj Low,Adj Close,Normalized Price,Daily Return,Prev Close,True Range,ATR
Date,,,,,,,,,
2025-06-23,3.40,3.45,3.34,3.43,1.000000,NaN,NaN,0.11,0.110000
2025-06-24,3.46,3.57,3.41,3.50,1.020408,0.020408,3.43,0.16,0.113571
2025-06-25,3.57,3.57,3.44,3.47,1.011662,-0.008571,3.50,0.13,0.114745
2025-06-26,3.46,3.48,3.40,3.42,0.997085,-0.014409,3.47,0.08,0.112263
2025-06-27,3.42,3.55,3.40,3.46,1.008746,0.011696,3.42,0.15,0.114959
...,...,...,...,...,...,...,...,...,...
2025-09-17,7.23,7.45,7.06,7.45,2.172012,0.061254,7.02,0.43,0.420248
2025-09-18,7.23,7.50,7.20,7.37,2.148688,-0.010738,7.45,0.30,0.411659
2025-09-19,7.59,7.60,7.28,7.37,2.148688,0.000000,7.37,0.32,0.405112


In [ ]:
_ticker = 'IREN'
_period = '3M'
_period_df = inspect_ticker_data(df_OHLCV, ticker=_ticker, period=_period)

### Sanity Check for Ticker: `IREN` | Period: `3M`

**1. Price Performance Calculation:**

   - Start Date (2025-06-23): 10.66
   - End Date   (2025-09-23): 41.77
   - Final Normalized Price: (41.77 / 10.66) = 3.9184



**2. Sharpe Ratio Calculation:**

   - Number of Trading Days: 65
   - Mean of Daily Returns: 0.022925
   - Std Dev of Daily Returns: 0.053319
   - Daily Sharpe Ratio: (0.022925 / 0.053319) = 0.4300
   - Annualized Sharpe Ratio: (0.4300 * sqrt(252)) = 6.8255



**3. Underlying Data Sample:**

,Adj Close,Daily Return
Date,,
2025-06-23,10.66,NaN
2025-06-24,11.54,0.082552
2025-06-25,11.87,0.028596
2025-06-26,13.11,0.104465
2025-06-27,14.00,0.067887


   ...


,Adj Close,Daily Return
Date,,
2025-09-17,37.90,0.039781
2025-09-18,36.32,-0.041689
2025-09-19,38.64,0.063877
2025-09-22,41.90,0.084369
2025-09-23,41.77,-0.003103


In [15]:
_ticker = 'IREN'
_period = '3M'
_period_df = inspect_ticker_data(df_OHLCV, ticker=_ticker, period=_period)

### Sanity Check for Ticker: `IREN` | Period: `3M`

**1. Price Performance Calculation:**

   - Start Date (2025-06-23): 10.66
   - End Date   (2025-09-23): 41.77
   - Final Normalized Price: (41.77 / 10.66) = 3.9184



**2. Sharpe Ratio Calculation:**

   - Number of Trading Days: 65
   - Mean of Daily Returns: 0.022925
   - Std Dev of Daily Returns: 0.053319
   - Annualized Sharpe Ratio: 6.8255



**3. Return/ATR Calculation:**

   - Mean of Daily Returns: 0.022925
   - Mean ATR (14-day EWM): 1.623931
   - Return / Mean ATR: 0.014117



**4. Underlying Data and Calculation Worksheet:**

,Adj Open,Adj High,Adj Low,Adj Close,Normalized Price,Daily Return,Prev Close,True Range,ATR
Date,,,,,,,,,
2025-06-23,10.720,10.820,9.825,10.66,1.000000,NaN,NaN,0.995,0.995000
2025-06-24,11.070,11.620,10.920,11.54,1.082552,0.082552,10.66,0.960,0.992500
2025-06-25,11.980,12.235,11.750,11.87,1.113508,0.028596,11.54,0.695,0.971250
2025-06-26,11.890,13.295,11.600,13.11,1.229831,0.104465,11.87,1.695,1.022946
2025-06-27,13.390,14.290,13.090,14.00,1.313321,0.067887,13.11,1.200,1.035593
...,...,...,...,...,...,...,...,...,...
2025-09-17,35.755,38.475,34.620,37.90,3.555347,0.039781,36.45,3.855,2.462469
2025-09-18,38.380,38.500,36.030,36.32,3.407129,-0.041689,37.90,2.470,2.463007
2025-09-19,36.710,39.870,36.630,38.64,3.624765,0.063877,36.32,3.550,2.540649


In [9]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

def inspect_ticker_data(df_ohlcv, ticker, period):
    """
    Performs a detailed calculation sanity check for a single ticker over a given period.

    Args:
        df_ohlcv (pd.DataFrame): The original 'long' format DataFrame with a 
                                 (Ticker, Date) MultiIndex.
        ticker (str): The ticker symbol to inspect (e.g., 'NVDA').
        period (str): The period to analyze (e.g., '1Y', '3M', '5D').
    """
    # --- 1. Data Preparation ---
    
    # Check if the ticker exists
    if ticker not in df_ohlcv.index.get_level_values(0):
        print(f"Error: Ticker '{ticker}' not found in the DataFrame.")
        return

    # Select all data for the specified ticker and create a clean DataFrame
    ticker_df = df_ohlcv.loc[ticker].copy()
    ticker_df.index = pd.to_datetime(ticker_df.index)
    
    # Define the start date based on the period
    end_date = ticker_df.index.max()
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    
    if period not in periods:
        print(f"Error: Period '{period}' is not a valid option. Choose from {list(periods.keys())}")
        return
        
    start_date = periods[period]
    
    # Slice the DataFrame for the requested period
    period_df = ticker_df.loc[start_date:end_date].copy()

    if len(period_df) < 2:
        print(f"Not enough data for ticker '{ticker}' in period '{period}' to perform calculations.")
        return

    # --- 2. Perform and Display Calculations ---

    display(Markdown(f"### Sanity Check for Ticker: `{ticker}` | Period: `{period}`"))

    # Price Performance
    start_price = period_df['Adj Close'].bfill().iloc[0]
    end_price = period_df['Adj Close'].ffill().iloc[-1]
    normalized_price = end_price / start_price
    
    display(Markdown("**1. Price Performance Calculation:**"))
    print(f"   - Start Date ({period_df.index.min().date()}): {start_price:,.2f}")
    print(f"   - End Date   ({period_df.index.max().date()}): {end_price:,.2f}")
    print(f"   - Final Normalized Price: ({end_price:,.2f} / {start_price:,.2f}) = {normalized_price:.4f}\n")

    # Sharpe Ratio
    period_df['Daily Return'] = period_df['Adj Close'].pct_change()
    mean_daily_return = period_df['Daily Return'].mean()
    std_daily_return = period_df['Daily Return'].std()
    
    # Handle case where standard deviation is zero (flat price)
    daily_sharpe = 0 if std_daily_return == 0 else mean_daily_return / std_daily_return
    annualized_sharpe = daily_sharpe * np.sqrt(252)

    display(Markdown("**2. Sharpe Ratio Calculation:**"))
    print(f"   - Number of Trading Days: {len(period_df)}")
    print(f"   - Mean of Daily Returns: {mean_daily_return:.6f}")
    print(f"   - Std Dev of Daily Returns: {std_daily_return:.6f}")
    print(f"   - Daily Sharpe Ratio: ({mean_daily_return:.6f} / {std_daily_return:.6f}) = {daily_sharpe:.4f}")
    print(f"   - Annualized Sharpe Ratio: ({daily_sharpe:.4f} * sqrt(252)) = {annualized_sharpe:.4f}\n")
    
    # --- 3. Display Underlying Data ---
    display(Markdown("**3. Underlying Data Sample:**"))
    display(period_df[['Adj Close', 'Daily Return']].head())
    if len(period_df) > 10:
        print("   ...")
        display(period_df[['Adj Close', 'Daily Return']].tail())
        
    return period_df[['Adj Close', 'Daily Return']]



In [10]:
_ticker = 'IREN'
_period = '3M'
_period_df = inspect_ticker_data(df_OHLCV, ticker=_ticker, period=_period)

### Sanity Check for Ticker: `IREN` | Period: `3M`

**1. Price Performance Calculation:**

   - Start Date (2025-06-23): 10.66
   - End Date   (2025-09-23): 41.77
   - Final Normalized Price: (41.77 / 10.66) = 3.9184



**2. Sharpe Ratio Calculation:**

   - Number of Trading Days: 65
   - Mean of Daily Returns: 0.022925
   - Std Dev of Daily Returns: 0.053319
   - Daily Sharpe Ratio: (0.022925 / 0.053319) = 0.4300
   - Annualized Sharpe Ratio: (0.4300 * sqrt(252)) = 6.8255



**3. Underlying Data Sample:**

,Adj Close,Daily Return
Date,,
2025-06-23,10.66,NaN
2025-06-24,11.54,0.082552
2025-06-25,11.87,0.028596
2025-06-26,13.11,0.104465
2025-06-27,14.00,0.067887


   ...


,Adj Close,Daily Return
Date,,
2025-09-17,37.90,0.039781
2025-09-18,36.32,-0.041689
2025-09-19,38.64,0.063877
2025-09-22,41.90,0.084369
2025-09-23,41.77,-0.003103


In [11]:
_ticker_df = df_OHLCV.loc[_ticker].copy()

##################

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pprint # Import the pretty-print library

def plot_interactive_performers_widget(df_ohlcv, 
                                       default_metric='Price',
                                       default_period='1Y', 
                                       default_rank_start=1,
                                       default_rank_end=10):
    """
    Creates a robust interactive plot using ipywidgets for state management.
    Allows for selecting tickers by a slice of their rank (e.g., 10th to 20th).
    Returns a list that is updated with the currently displayed tickers.
    """
    # --- [Section 1 & 2: Data Prep & Pre-computation] ---
    print("Reshaping and pre-computing all performance data...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex) or 'Adj Close' not in df_ohlcv.columns:
        raise ValueError("Expected 'long' format with (Ticker, Date) MultiIndex and 'Adj Close' column.")
    df_close = df_ohlcv['Adj Close'].unstack(level=0)
    df_close.index = pd.to_datetime(df_close.index)
    end_date = df_close.index.max()
    metrics = ['Price', 'Sharpe']
    periods = {
        '1D': end_date - pd.DateOffset(days=1), '5D': end_date - pd.DateOffset(days=5),
        '3M': end_date - pd.DateOffset(months=3), '6M': end_date - pd.DateOffset(months=6),
        'YTD': datetime(end_date.year, 1, 1), '1Y': end_date - pd.DateOffset(years=1),
        '5Y': end_date - pd.DateOffset(years=5),
    }
    all_rankings = {metric: {} for metric in metrics}
    period_normalized_data = {}
    for label, start_date in periods.items():
        period_df = df_close.loc[start_date:end_date]
        if period_df.empty or len(period_df) < 2: continue
        first_prices, last_prices = period_df.bfill().iloc[0], period_df.ffill().iloc[-1]
        all_rankings['Price'][label] = (last_prices / first_prices).dropna().sort_values(ascending=False).index.tolist()
        daily_returns = period_df.pct_change()
        mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
        sharpe_ratio = (mean_returns / std_returns).fillna(0)
        all_rankings['Sharpe'][label] = (sharpe_ratio * np.sqrt(252)).sort_values(ascending=False).index.tolist()
        period_normalized_data[label] = period_df.div(first_prices)
    print("Pre-computation finished.")

    # --- 3. Create ipywidgets Controls & Data Holder ---
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    max_traces = 50 
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    period_dropdown = widgets.Dropdown(options=list(periods.keys()), value=default_period, description='Period:')
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    
    _currently_displayed_tickers = []

    # --- 4. Create a FigureWidget ---
    fig = go.FigureWidget()
    for i in range(max_traces):
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))

    # --- 5. Define the Update Logic ---
    def update_plot(button_click):
        metric, period, rank_start, rank_end = metric_dropdown.value, period_dropdown.value, rank_start_dropdown.value, rank_end_dropdown.value
        
        ticker_list_output.clear_output()
        
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be less than or equal to 'Rank End'.")
            return
        
        ranked_tickers = all_rankings[metric].get(period, [])
        tickers_to_display = ranked_tickers[rank_start-1:rank_end]
        norm_data = period_normalized_data.get(period)

        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(tickers_to_display) and norm_data is not None:
                    ticker = tickers_to_display[i]
                    trace.x, trace.y, trace.name = norm_data[ticker].index, norm_data[ticker], ticker
                    trace.visible, trace.showlegend = True, True
                else:
                    trace.visible, trace.showlegend = False, False
        
        _currently_displayed_tickers.clear()
        _currently_displayed_tickers.extend(tickers_to_display)
        
        with ticker_list_output:
            print("Currently Displayed Tickers:")
            # --- NEW: Use compact=True to force a more horizontal layout ---
            pprint.pprint(tickers_to_display, width=120, compact=True) 

    # --- 6. Set up Initial View and Layout ---
    update_plot(None)
    fig.update_layout(
        title_text='Top Performing Tickers (Normalized to 1)',
        xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)',
        hovermode='x unified', legend_title_text='Tickers',
        height=700, margin=dict(t=50)
    )
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    
    # --- 7. Display the UI ---
    controls = widgets.HBox([metric_dropdown, period_dropdown, rank_start_dropdown, rank_end_dropdown, update_button])
    ui_container = widgets.VBox([controls, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    
    display(ui_container, fig)
    
    return _currently_displayed_tickers

In [ ]:
# --- NEW: Initialize the variable before the call ---
plotted_tickers = [] 

# Call the final function and capture the returned list
# The 'plotted_tickers' list will be updated in place every time you click the button.
plotted_tickers = plot_interactive_performers_widget(
    df_ohlcv=df_OHLCV, 
    default_metric='Sharpe', 
    default_period='1Y', 
    default_rank_start=20, 
    default_rank_end=40
)

print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
print(plotted_tickers)

In [ ]:
print("\n--- After the plot, the 'plotted_tickers' variable holds the last displayed list: ---")
print(plotted_tickers)